In [1]:
import pandas as pd
import os
from pathlib import Path

# ExcelDataProcessor Class Documentation

## Overview
The `ExcelDataProcessor` class is designed to process Excel files containing data with Persian calendar dates. It provides functionality to read multiple Excel files, extract specific cell values, and organize the data into a sorted DataFrame based on Persian calendar dates.

## Key Features
- Reads both regular Excel files (.xls, .xlsx) and HTML-formatted Excel files
- Handles Persian calendar dates in column names
- Converts string values to float type
- Supports tuple-based column names (common in multi-index Excel files)
- Sorts data based on Persian calendar dates

## Class Methods

### `__init__(data_directory='data')`
Initializes the processor with a specified data directory.
- **Parameters:**
  - `data_directory` (str): Path to the directory containing Excel files (default: 'data')

### `read_excel_files()`
Reads all Excel files from the specified directory.
- **Returns:**
  - `bool`: True if files were successfully read, False otherwise

### `get_cell_values(row_idx, col_idx)`
Extracts values from specific row and column indices across all files.
- **Parameters:**
  - `row_idx` (int): Row index
  - `col_idx` (int): Column index
- **Returns:**
  - `pd.DataFrame`: DataFrame with columns:
    - Filename: Name of the Excel file
    - Value: Numeric value from the specified cell
    - Date: Formatted Persian date
    - Column_Detail: Second part of tuple column name (if available)

## Usage Example
```python
# Create an instance
processor = ExcelDataProcessor(data_directory='data')

# Read all Excel files
if processor.read_excel_files():
    # Get values from row 1, column 13
    result_df = processor.get_cell_values(1, 13)
    print(result_df)
```

## Data Format
The resulting DataFrame will be sorted by Persian calendar dates and include:
- Files with valid dates sorted in chronological order
- Files with unknown dates placed at the end
- All numeric values converted to float type
- Additional column details when available

In [5]:
class ExcelDataProcessor:
    def __init__(self, data_directory='data'):
        """
        Initialize ExcelDataProcessor with a data directory path.
        
        Args:
            data_directory (str): Path to the directory containing Excel files
        """
        self.data_directory = data_directory
        self.excel_data = {}
        self._base_path = Path(os.getcwd()).parent if os.path.basename(os.getcwd()) == 'estimator' else Path(os.getcwd())
        self._data_path = self._base_path / data_directory
        
    def read_excel_files(self):
        """
        Read all Excel files in the specified directory.
        Returns:
            bool: True if files were read successfully, False otherwise
        """
        try:
            # Iterate through all files in the directory
            for file_path in self._data_path.glob('*.xls*'):
                try:
                    try:
                        # First try to read as a regular Excel file
                        df = pd.read_excel(
                            file_path,
                            engine='openpyxl' if file_path.suffix == '.xlsx' else 'xlrd'
                        )
                    except Exception as excel_error:
                        # If that fails, try to read as HTML
                        print(f"Attempting to read {file_path.name} as HTML...")
                        df = pd.read_html(
                            file_path,
                            encoding='utf-8',
                            flavor='lxml'
                        )[0]
                    
                    if not df.empty:
                        self.excel_data[file_path.name] = df
                        print(f"Successfully read {file_path.name}")
                    else:
                        print(f"Warning: {file_path.name} is empty")
                        
                except Exception as e:
                    print(f"Error reading {file_path.name}: {str(e)}")
            
            return len(self.excel_data) > 0
            
        except Exception as e:
            print(f"Error accessing directory: {str(e)}")
            return False
    
    def get_cell_values(self, row_idx, col_idx):
        """
        Get values from specific row and column indices across all files.
        
        Args:
            row_idx (int): Row index
            col_idx (int): Column index
            
        Returns:
            pd.DataFrame: DataFrame with filename, value, date information, and column details
        """
        values = []
        dates = []
        filenames = []
        column_names = []  # For the second part of the tuple
        
        for filename, df in self.excel_data.items():
            try:
                # Get value and convert to float
                value = df.iloc[row_idx, col_idx]
                if isinstance(value, str):
                    value = ''.join(c for c in value if c.isdigit() or c == '.')
                    value = float(value) if value else 0.0
                else:
                    value = float(value) if pd.notna(value) else 0.0
                
                # Extract date from column name (first part of tuple)
                col_name = df.columns[col_idx]
                if isinstance(col_name, tuple):
                    date_part = col_name[0]  # First part for date
                    col_detail = col_name[1]  # Second part for additional info
                else:
                    date_part = col_name
                    col_detail = "Unknown"
                
                formatted_date = self._extract_persian_date(date_part)
                
                filenames.append(filename)
                values.append(value)
                dates.append(formatted_date)
                column_names.append(col_detail)
                
            except (IndexError, ValueError) as e:
                print(f"Error processing {filename}: {str(e)}")
                continue
        
        return self._create_sorted_dataframe(filenames, values, dates, column_names)
    
    def _extract_persian_date(self, col_name):
        """
        Extract and format Persian date from column name.
        
        Args:
            col_name (str): Column name containing date
            
        Returns:
            str: Formatted date or 'Unknown'
        """
        try:
            import re
            date_match = re.search(r'\d{2,4}[/\-]\d{1,2}[/\-]\d{1,2}', str(col_name))
            if date_match:
                date = date_match.group()
                parts = date.split('/')
                if len(parts[0]) == 2:
                    parts[0] = '14' + parts[0] if int(parts[0]) < 50 else '13' + parts[0]
                return f"{parts[0]}/{parts[1].zfill(2)}/{parts[2].zfill(2)}"
        except Exception:
            pass
        return "Unknown"
    
    def _create_sorted_dataframe(self, filenames, values, dates, column_names):
        """
        Create and sort DataFrame by Persian calendar dates.
        
        Args:
            filenames (list): List of filenames
            values (list): List of values
            dates (list): List of dates
            column_names (list): List of column second parts
            
        Returns:
            pd.DataFrame: Sorted DataFrame
        """
        df = pd.DataFrame({
            'Filename': filenames,
            'Value': values,
            'Date': dates,
            'Column_Detail': column_names
        })
        
        # Create sorting key
        df['SortKey'] = df['Date'].apply(lambda x: float('inf') if x == 'Unknown' 
                                        else int(''.join(x.split('/'))))
        
        # Sort and clean up
        df_sorted = df.sort_values(by='SortKey', ascending=True)
        df_sorted = df_sorted.drop('SortKey', axis=1)
        return df_sorted.reset_index(drop=True)



### Example --- read file data 

In [6]:
# Create an instance of ExcelDataProcessor
processor = ExcelDataProcessor(data_directory='data')

# Read all Excel files
if processor.read_excel_files():
    # Get values from row 1, column 13 (same as before)
    result_df = processor.get_cell_values(1, 13)


Attempting to read 0013efe6-cd1a-4023-8a47-152850eebefc.xls as HTML...
Successfully read 0013efe6-cd1a-4023-8a47-152850eebefc.xls
Attempting to read 00b9b5c8-00ac-44b1-a9ae-8156ab5f63dd.xls as HTML...
Successfully read 00b9b5c8-00ac-44b1-a9ae-8156ab5f63dd.xls
Attempting to read 0558e64e-5345-4c65-8fc0-991770603431.xls as HTML...
Successfully read 0558e64e-5345-4c65-8fc0-991770603431.xls
Attempting to read 057ff4ea-d363-41e7-9ffa-34b48ed1246e.xls as HTML...
Successfully read 057ff4ea-d363-41e7-9ffa-34b48ed1246e.xls
Attempting to read 08a39594-ad71-40dd-b8cc-cc691083516d.xls as HTML...
Successfully read 08a39594-ad71-40dd-b8cc-cc691083516d.xls
Attempting to read 12264d38-3f6e-471f-9d57-051f0009aae9.xls as HTML...
Successfully read 12264d38-3f6e-471f-9d57-051f0009aae9.xls
Attempting to read 1b04c298-6fbe-4ba3-b466-62f317e16065.xls as HTML...
Successfully read 1b04c298-6fbe-4ba3-b466-62f317e16065.xls
Attempting to read 1c2258b8-9b36-45f2-813f-3567e928123a.xls as HTML...
Successfully read 1

In [8]:
result_df.to_csv('output.csv', index=False)
result_df

,Filename,Value,Date,Column_Detail
0,c1fe2967-6a45-4469-b203-cd289100bd57.xls,13411.00,۱۴۰۰/۰۵/۳۱,تعداد تولید
1,1b04c298-6fbe-4ba3-b466-62f317e16065.xls,15175.00,۱۴۰۰/۰۶/۳۱,تعداد تولید
2,5624b460-85f3-43c6-b16a-1cadc3ab230f.xls,16221.95,۱۴۰۰/۰۷/۳۰,تعداد تولید
3,057ff4ea-d363-41e7-9ffa-34b48ed1246e.xls,17585.51,۱۴۰۰/۰۸/۳۰,تعداد تولید
4,c3b89a68-0e30-48ca-8b81-f902ee2d121d.xls,20169.00,۱۴۰۰/۰۹/۳۰,تعداد تولید
5,8cac28c5-3e6e-4bce-9b4b-c0b0a4784d29.xls,18755.00,۱۴۰۰/۱۰/۳۰,تعداد تولید
6,e48085cc-9f12-4ea1-9334-371ae86a1a88.xls,16409.00,۱۴۰۰/۱۱/۳۰,تعداد تولید
7,76eb5d39-e81b-4cc1-b4fe-4c4a667e7a98.xls,15505.54,۱۴۰۰/۱۲/۲۹,تعداد تولید
8,9b83b316-8caf-4ee3-8cdd-8d542ce522db.xls,15505.54,۱۴۰۰/۱۲/۲۹,تعداد تولید
9,33712f03-28ca-4140-b836-00da55faa92c.xls,490496.00,۱۴۰۱/۰۲/۳۱,تعداد تولید
